## Configuration

In [0]:
storage_account_name = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  
storage_account_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" 
container_name = "raw"
# On configure l'accès pour ce notebook
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_key
)

print("🔌 Connexion configurée.")



🔌 Connexion configurée.


## 2. LECTURE & NETTOYAGE POLLUTION (RAW)

In [0]:
from pyspark.sql.functions import col, to_timestamp, date_trunc, lit
path_pol = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/history_pm25*.json"
df_pol_raw = spark.read.option("multiline", "true").json(path_pol)

df_pollution_clean = df_pol_raw.select(
    date_trunc("hour", to_timestamp(col("period.datetimeFrom.utc"))).alias("date_heure"),
    col("value").cast("double").alias("pm25_value"),
    lit("Le Havre").alias("ville"),
    lit("14126423").alias("sensor_id")
)
df_pollution_clean.show()

+-------------------+----------+--------+---------+
|         date_heure|pm25_value|   ville|sensor_id|
+-------------------+----------+--------+---------+
|2025-09-13 09:00:00|       2.8|Le Havre| 14126423|
|2025-09-14 10:00:00|       3.5|Le Havre| 14126423|
|2025-09-14 14:00:00|       0.9|Le Havre| 14126423|
|2025-09-15 06:00:00|       9.3|Le Havre| 14126423|
|2025-09-15 09:00:00|       8.5|Le Havre| 14126423|
|2025-09-15 10:00:00|       8.7|Le Havre| 14126423|
|2025-09-15 20:00:00|       9.9|Le Havre| 14126423|
|2025-09-15 21:00:00|       9.9|Le Havre| 14126423|
|2025-09-15 22:00:00|      10.0|Le Havre| 14126423|
|2025-09-15 23:00:00|       9.8|Le Havre| 14126423|
|2025-09-16 00:00:00|       9.6|Le Havre| 14126423|
|2025-09-16 01:00:00|       8.6|Le Havre| 14126423|
|2025-09-16 02:00:00|       8.7|Le Havre| 14126423|
|2025-09-16 03:00:00|       8.4|Le Havre| 14126423|
|2025-09-16 04:00:00|       8.4|Le Havre| 14126423|
|2025-09-16 05:00:00|       8.2|Le Havre| 14126423|
|2025-09-16 

## 3. LECTURE & NETTOYAGE MÉTÉO (RAW)

In [0]:
path_meteo = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/weather_full*.json"
df_meteo_raw = spark.read.option("multiline", "true").json(path_meteo)


df_meteo_clean = df_meteo_raw.select(
    to_timestamp(col("date_heure")).alias("date_heure"),
    col("temperature").cast("double"),
    col("humidite").cast("double"),
    col("vent_vitesse").cast("double"),
    col("vent_direction").cast("double"),
    col("precipitation").cast("double"),
    col("type") 
).dropDuplicates(["date_heure"]).orderBy("date_heure")

print(f"✅ Météo chargée (Passé + Futur 15j) : {df_meteo_clean.count()} lignes")
display(df_meteo_clean)

✅ Météo chargée (Passé + Futur 15j) : 3168 lignes


date_heure,temperature,humidite,vent_vitesse,vent_direction,precipitation,type
2025-09-19T00:00:00Z,15.6,94.0,12.6,124.0,0.0,null
2025-09-19T01:00:00Z,15.6,93.0,10.4,134.0,0.0,null
2025-09-19T02:00:00Z,14.8,93.0,10.9,123.0,0.0,null
2025-09-19T03:00:00Z,14.5,92.0,10.4,124.0,0.0,null
2025-09-19T04:00:00Z,14.4,91.0,11.9,124.0,0.0,null
2025-09-19T05:00:00Z,14.4,90.0,13.2,124.0,0.0,null
2025-09-19T06:00:00Z,14.2,89.0,14.1,120.0,0.0,null
2025-09-19T07:00:00Z,14.1,87.0,14.3,118.0,0.0,null
2025-09-19T08:00:00Z,14.9,84.0,14.5,121.0,0.0,null
2025-09-19T09:00:00Z,16.5,83.0,13.5,109.0,0.0,null


## 4. FUSION (JOINTURE)

In [0]:
print("🔄 Fusion des deux sources...")
df_silver = df_meteo_clean.join(
    df_pollution_clean,
    on="date_heure",
    how="left"
).orderBy("date_heure")

🔄 Fusion des deux sources...


## SAUVEGARDE DANS CLEAN (SILVER)

In [0]:
path_silver = f"abfss://clean@{storage_account_name}.dfs.core.windows.net/silver_le_havre_complete"
df_silver.write.format("delta").mode("overwrite").save(path_silver)

print(f"✅ SUCCÈS ! Table Silver créée : {path_silver}")
display(df_silver)

✅ SUCCÈS ! Table Silver créée : abfss://clean@sadatalakeair2025.dfs.core.windows.net/silver_le_havre_complete


date_heure,temperature,humidite,vent_vitesse,vent_direction,precipitation,type,pm25_value,ville,sensor_id
2025-09-19T00:00:00Z,15.6,94.0,12.6,124.0,0.0,null,null,null,null
2025-09-19T01:00:00Z,15.6,93.0,10.4,134.0,0.0,null,3.2,Le Havre,14126423
2025-09-19T02:00:00Z,14.8,93.0,10.9,123.0,0.0,null,3.7,Le Havre,14126423
2025-09-19T03:00:00Z,14.5,92.0,10.4,124.0,0.0,null,5.1,Le Havre,14126423
2025-09-19T04:00:00Z,14.4,91.0,11.9,124.0,0.0,null,5.6,Le Havre,14126423
2025-09-19T05:00:00Z,14.4,90.0,13.2,124.0,0.0,null,5.8,Le Havre,14126423
2025-09-19T06:00:00Z,14.2,89.0,14.1,120.0,0.0,null,5.5,Le Havre,14126423
2025-09-19T07:00:00Z,14.1,87.0,14.3,118.0,0.0,null,5.6,Le Havre,14126423
2025-09-19T08:00:00Z,14.9,84.0,14.5,121.0,0.0,null,7.0,Le Havre,14126423
2025-09-19T09:00:00Z,16.5,83.0,13.5,109.0,0.0,null,6.4,Le Havre,14126423
